https://www.cnblogs.com/hellcat/p/8565340.html  
ufunc是universal function的缩写，它是一种能对数组的每个元素进行操作的函数。NumPy内置的许多ufunc函数都是在C语言级别实现的，因此它们的计算速度非常快。  
通过组合标准的ufunc函数的调用，可以实现各种算式的数组计算。不过有些时候这种算式不易编写，而针对每个元素的计算函数却很容易用Python实现，这时可以用frompyfunc函数将一个计算单个元素的函数转换成ufunc函数。这样就可以方便地用所产生的ufunc函数对数组进行计算了。

In [18]:
def f(x):
    ''' 生成一个单个元素映射的函数'''
    if x==1 or x==0:
        return x
    else:
        return f(x-1)+f(x-2)


In [10]:
x = np.arange(1, 30)
# f(x) # 会报错

In [11]:
time [f(i) for i in x]

Wall time: 2.45 s


[1,
 1,
 2,
 3,
 5,
 8,
 13,
 21,
 34,
 55,
 89,
 144,
 233,
 377,
 610,
 987,
 1597,
 2584,
 4181,
 6765,
 10946,
 17711,
 28657,
 46368,
 75025,
 121393,
 196418,
 317811,
 514229]

In [17]:
F = np.frompyfunc( lambda x: f(x), 1, 1)
%time F(x)

Wall time: 0 ns
Wall time: 635 ms


array([1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987,
       1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393,
       196418, 317811, 514229], dtype=object)

或者，写得好看些，再加点参数

In [91]:
%%timeit 
def F(a1, a2, x):
    def f(a1, a2, x):
        return np.sin(a1*x)+a2
    return np.frompyfunc(f,3,1)(a1, a2, x)

x = np.linspace(0,10,10000,dtype=object)
F(3, 20, x)

10.9 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [92]:
%%timeit 
def _f(a1, a2, x):
    return np.sin(a1*x)+a2
x = np.linspace(0,10,10000)
res = [_f(3,20,i) for i in x]


18.1 ms ± 597 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [79]:
%%time
def f(a1, a2, x):
    return np.sin(a1*x)+a2

x = np.linspace(0,10,100000)
f(3, 2, x)

Wall time: 997 µs


array([2.        , 2.0003    , 2.00060001, ..., 1.011876  , 1.01192214,
       1.01196838])

# 写成装饰器的形式：


In [111]:
def broadcast(func):
    def wrap(*args, **kwargs):
        '''
        Takes an arbitrary Python function and returns a NumPy ufunc
        Can be used, for example, to add broadcasting to a built-in Python function
        return: only one out, 
        type:numpy object
        '''
        nin, nout = len(args)+len(kwargs), 1
        return np.frompyfunc(func,nin, nout)(*args, **kwargs)
    return wrap

In [116]:
@broadcast
def f(x):
    ''' 一个只能单个元素映射的函数'''
    if x==1 or x==0:
        return x
    else:
        return f(x-1)+f(x-2)
    
f([2,4,10])

array([1, 3, 832040], dtype=object)

In [121]:
@broadcast
def g(a1, a2, x):
    if a1<a2:
        a1=a2
    return a1+a2*x

g(1,5, 9)

50

In [122]:
g(1,5, [2,5,9])

array([15, 30, 50], dtype=object)